In [1]:
import matplotlib.pyplot as plt
from scipy import signal
import math
import time
from interp3d import interp_3d
from scipy.interpolate import RegularGridInterpolator
import rasterio
import os
import numpy as np
import tqdm
import glob
from pyproj import Proj,transform
import matplotlib.gridspec as gridspec
import py3dep
from shapely.geometry import Polygon

In [2]:
files = glob.glob(os.getcwd()+'/*aggregate_3d_dsm.tif')
print(files)

['/scratch/e.conway/3DReconstruction/Jacksonville/Analysis/4_aggregate_3d_dsm.tif', '/scratch/e.conway/3DReconstruction/Jacksonville/Analysis/3_aggregate_3d_dsm.tif', '/scratch/e.conway/3DReconstruction/Jacksonville/Analysis/2_aggregate_3d_dsm.tif', '/scratch/e.conway/3DReconstruction/Jacksonville/Analysis/1_aggregate_3d_dsm.tif']


In [3]:
files = np.array(files)
nfiles = len(files)

In [4]:
# Step 1 - make a large 2d array of lon and lat, for which we have coverage
# nfiles = n limits of lon/lat

minima_lat = np.zeros(nfiles)
minima_lon = np.zeros(nfiles)
maxima_lat = np.zeros(nfiles)
maxima_lon = np.zeros(nfiles)
file_order = np.zeros(nfiles)
raw_lon = np.ndarray(shape=(nfiles),dtype=np.object_)#np.zeros((200,nfiles))
raw_lat = np.ndarray(shape=(nfiles),dtype=np.object_)#np.zeros((200,nfiles))

func = np.ndarray(shape=(nfiles),dtype=np.object_)

In [5]:
for i in range(nfiles):
    with rasterio.open(files[i],'r') as df:
        band = df.read()
        band = band.transpose((1,2,0))
        band = band[::-1,:,:]
        crs = df.crs
        bounds = df.bounds
        #print(bounds)
        #print(crs)
        trans = df.transform
        inProj =  Proj(crs)
        outProj = Proj('epsg:4326')
        y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))

        maxima_lon[i] = x1[1]
        minima_lon[i] = x1[0]
        maxima_lat[i] = y1[1]
        minima_lat[i] = y1[0]
        
        raw_lon[i] = np.linspace(minima_lon[i],maxima_lon[i],band.shape[1])
        raw_lat[i] = np.linspace(minima_lat[i],maxima_lat[i],band.shape[0])[::-1]
        
    t = np.ones((band.shape))
    iband = np.concatenate((band,t),axis=2)
    iband = np.concatenate((iband,t),axis=2)
    iband[:,:,1] = iband[:,:,0]
    
    z = np.ones(3)
    z[0] = 0
    z[1] = 1
    z[2] = 2

    func[i] = interp_3d.Interp3D(iband,raw_lat[i],raw_lon[i],z)
    #func[i] = RegularGridInterpolator((raw_lat[i],raw_lon[i]),band)
    

/tmp/ipykernel_2232/3023474311.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
/tmp/ipykernel_2232/3023474311.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
/tmp/ipykernel_2232/3023474311.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
/tmp/ipykernel_2232/3023474311.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to

In [12]:
nlon=0
nlat=0
lon=[]
lat=[]
for i in range(nfiles):
    if(i==0):
        lat.append(raw_lat[i])
        lon.append(raw_lon[i])
        lat=np.array(lat)
        lon=np.array(lon)
        lon = lon.flatten()
        lat = lat.flatten()
    else:
        lat = np.concatenate((lat,raw_lat[i]))
        lon = np.concatenate((lon,raw_lon[i]))



lat = np.linspace(np.min(lat) , np.max(lat), 3200 )
lon = np.linspace(np.min(lon) , np.max(lon), 3200 )

In [13]:
elev = np.zeros((len(lat),len(lon)))
for i in tqdm.tqdm(range(len(lon))):
    #print(i)
    for k in range(len(lat)):
        done=False
        for j in range(nfiles):
            if( (lon[i] <= maxima_lon[j] ) and (lat[k] <= maxima_lat[j]) and (lon[i] >= minima_lon[j] ) and (lat[k] >= minima_lat[j]) ):
                elev[k,i] = func[j]((lat[k],lon[i],0.0000))
                done=True
                #if(elev[k,i] < -1000 ):
                #    elev[k,i] = np.nan


100%|██████████| 3200/3200 [01:14<00:00, 43.03it/s]


In [14]:
usgs = '3DEP_USGS10m.tif'
with rasterio.open(usgs,'r') as df:
        lidar = df.read()
        lidar = lidar.transpose((1,2,0))
        lidar = lidar[::-1,:,:]
        crs = df.crs
        bounds = df.bounds
        trans = df.transform
        inProj =  Proj(crs)
        outProj = Proj('epsg:4326')
        y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
        
        lon_lidar = np.linspace(y1[0],y1[1],lidar.shape[1])
        lat_lidar = np.linspace(x1[0],x1[1],lidar.shape[0])
        
        t = np.ones((lidar.shape))
        iband = np.concatenate((lidar[:,:,:],t),axis=2)
        iband = np.concatenate((iband,t),axis=2)
        iband[:,:,1] = iband[:,:,0]

        z = np.ones(3)
        z[0] = 0
        z[1] = 1
        z[2] = 2

        lidar_interp = interp_3d.Interp3D(iband,lat_lidar,lon_lidar,z)
        

/tmp/ipykernel_2232/4170206672.py:11: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))


In [15]:
lidar_new = np.zeros((len(lat),len(lon)))

for i in tqdm.tqdm(range(len(lat))):
    for j in range(len(lon)):
        lidar_new[i,j] = lidar_interp((lat[i],lon[j],1))

100%|██████████| 3200/3200 [00:21<00:00, 148.06it/s]


In [18]:
idx = np.argsort(lon)
lon=lon[idx]
elev=elev[:,idx]
idx = np.argsort(lat)
lat=lat[idx]
elev=elev[idx,:]

fig=plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2,figure=fig)


with rasterio.open(files[1],'r') as df:
        band = df.read()
        band = band.transpose((1,2,0))
        crs = df.crs
        bounds = df.bounds
        #print(bounds)
        #print(crs)
        trans = df.transform
        inProj =  Proj(crs)
        outProj = Proj('epsg:4326')
        y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))

ax = plt.subplot(gs[0, 0])
ax.set_title('Tile (A)')
ax.set_ylabel(r'Latitude [$^{o}$]')
ax.set_xlabel(r'Longitude [$^{o}$]')
sc=ax.imshow(band[::-1,:,0],aspect='auto',origin='lower',extent=[np.min(x1),np.max(x1),np.min(y1),np.max(y1)],vmin=-50,vmax=10)
plt.colorbar(sc,label='Elevation Above Ground [m]')


with rasterio.open(files[0],'r') as df:
        band = df.read()
        band = band.transpose((1,2,0))
        crs = df.crs
        bounds = df.bounds
        #print(bounds)
        #print(crs)
        trans = df.transform
        inProj =  Proj(crs)
        outProj = Proj('epsg:4326')
        y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
ax = plt.subplot(gs[0, 1])
ax.set_title('Tile (B)')
ax.set_xlabel(r'Longitude [$^{o}$]')
sc=ax.imshow(band[::-1,:,0],aspect='auto',origin='lower',extent=[np.min(x1),np.max(x1),np.min(y1),np.max(y1)],vmin=-50,vmax=10)
plt.colorbar(sc,label='Elevation Above Ground [m]')



#fig=plt.figure(figsize=(12,12))
ax = plt.subplot(gs[1, 0])
ax.set_title('Fusion of Tiles (A,B)')
ax.set_ylabel(r'Latitude [$^{o}$]')
ax.set_xlabel(r'Longitude [$^{o}$]')
sc=ax.imshow(elev[::-1,:],aspect='auto',origin='lower',extent=[np.min(lon),np.max(lon),np.min(lat),np.max(lat)],vmin=-50,vmax=10)
plt.colorbar(sc,label='Elevation Above Ground [m]')

ax = plt.subplot(gs[1, 1])
ax.set_title('3DEP 10m LiDAR')
ax.set_ylabel(r'Latitude [$^{o}$]')
ax.set_xlabel(r'Longitude [$^{o}$]')
sc=ax.imshow(lidar_new,aspect='auto',origin='lower',extent=[np.min(lon),np.max(lon),np.min(lat),np.max(lat)],vmin=0,vmax=3)
plt.colorbar(sc,label='Elevation Above Ground [m]')

#plt.show()
plt.savefig('test_fusion_with_lidar.jpeg',dpi=1000)
plt.close()

/tmp/ipykernel_2232/1732537199.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
/tmp/ipykernel_2232/1732537199.py:42: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  y1,x1 = transform(inProj,outProj,np.array([bounds[0],bounds[2]]),np.array([bounds[1],bounds[3]]))
